An example of open-domain question answering using DPR, not completed.  
Reference source code is from https://github.com/facebookresearch/DPR.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !wget https://dl.fbaipublicfiles.com/dpr/checkpoint/retriever/single/nq/hf_bert_base.cp
!wget https://dl.fbaipublicfiles.com/dpr/checkpoint/retriver/single-adv-hn/nq/hf_bert_base.cp
!wget https://dl.fbaipublicfiles.com/dpr/checkpoint/reader/nq-single/hf_bert_base.cp
!pip install transformers

In [1]:
import numpy as np
import tensorflow as tf
from transformers import BertTokenizerFast, DPRReaderTokenizer
from drive.MyDrive.Python.Research.modules import dpr

Build an end-to-end QA model with DPR.

In [ ]:
class ModelQA:
  def __init__(self, retriever, reader):
    self.vec, self.bs, self.topk, self.qlen, self.tlen, self.clen, self.alen = {}, 128, 5, 48, 16, 448, 32
    self.ctokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
    self.rtokenizer = DPRReaderTokenizer.from_pretrained('facebook/dpr-reader-single-nq-base')
    self.qencoder = dpr.DPR('question_model')
    self.qencoder.loading(retriever)
    self.cencoder = dpr.DPR('ctx_model')
    self.cencoder.loading(retriever)
    self.reader = dpr.Reader('encoder')
    self.reader.loading(reader)

  def context_encoding(self, context):
    @tf.function
    def encoding(w, s, m):
      return self.cencoder.propagating(w, s, m)[0][:, 0, :]

    inp, res = self.ctokenizer(
      context,
      max_length=self.clen,
      padding='max_length',
      truncation=True,
      stride=self.clen//2,
      return_overflowing_tokens=True,
      return_tensors='np'), []
    
    for word, seg, mask in tf.data.Dataset.from_tensor_slices((
      inp['input_ids'], inp['token_type_ids'], 1.-inp['attention_mask'])).batch(self.bs):
      res.append(encoding(word, seg, mask).numpy())
    
    return np.concatenate(res), list(inp['input_ids']), list(inp['overflow_to_sample_mapping'])
    
  def question_encoding(self, question):
    inp = self.ctokenizer(question, max_length=self.qlen, padding=True, truncation=True, return_tensors='np')
    res = self.qencoder.propagating(inp['input_ids'], inp['token_type_ids'], 1.-inp['attention_mask'])[0][:, 0, :]
    return res, inp['input_ids']
  
  def context_updating(self, context):
    title, self.vec = list(context.keys()), {}
    vec, self.vec['token'], mapping = self.context_encoding([context[idx] for idx in title])
    self.vec['title'] = [title[idx] for idx in mapping]
    self.vec['vector'] = tf.linalg.l2_normalize(vec, axis=-1)

  def context_searching(self, question):
    vec, token = self.question_encoding(question)
    matrix = tf.matmul(tf.linalg.l2_normalize(vec, axis=-1), self.vec['vector'], transpose_b=True)
    return list(np.argsort(matrix)[0])[::-1][:self.topk], token

  def answer_predicting(self, query):
    cand, token = self.context_searching(query)
    title = [self.ctokenizer(self.vec['title'][idx], max_length=self.tlen, truncation=True) for idx in cand]
    title = [self.ctokenizer.decode(idx['input_ids'], skip_special_tokens=True) for idx in title]
    span = [self.ctokenizer.decode(self.vec['token'][idx], skip_special_tokens=True) for idx in cand]
    question = [self.ctokenizer.decode(token[0], skip_special_tokens=True)]*self.topk
    inp = self.rtokenizer(questions=question, titles=title, texts=span, padding=True, truncation=True, return_tensors='np')
    cls, pos = self.reader.propagating(inp['input_ids'], tf.zeros_like(inp['input_ids']), 1.-inp['attention_mask'])
    start, end = tf.nn.softmax(pos[0][np.argmax(cls)]), tf.nn.softmax(pos[1][np.argmax(cls)])
    val, ans = 0., 'Sorry, cannot find an answer.'

    for s in range(start.shape[0]):
      for e in range(end.shape[0]):
        prob = start[s]*end[e]

        if s > e or e-s > self.alen or val > prob:
          continue
        else:
          token = inp['input_ids'][np.argmax(cls)][s:e+1]
          val, ans = prob, self.ctokenizer.decode(token, skip_special_tokens=True)

      if start[s] < val:
        continue

    return ans, val


qa_model = ModelQA('hf_bert_base.cp', 'hf_bert_base.cp.1')

Have a test, and context examples are from Wikipedia.

In [3]:
test_context = {
  "Apple Inc.": "Apple Inc. is an American multinational technology company that specializes in consumer electronics, computer software, and online services. Apple is the world's largest technology company by revenue (totaling $274.5 billion in 2020) and, since January 2021, the world's most valuable company. As of 2021, Apple is the world's fourth-largest PC vendor by unit sales, and fourth-largest smartphone manufacturer. It is one of the Big Five American information technology companies, along with Amazon, Google, Microsoft, and Facebook.",
  "Formula One": "Formula One (also known as Formula 1 or F1) is the highest class of international auto racing for single-seater formula racing cars sanctioned by the Fédération Internationale de l'Automobile (FIA). The World Drivers' Championship, which became the FIA Formula One World Championship in 1981, has been one of the premier forms of racing around the world since its inaugural season in 1950. The word formula in the name refers to the set of rules to which all participants' cars must conform. A Formula One season consists of a series of races, known as Grands Prix, which take place worldwide on both purpose-built circuits and closed public roads.",
  "Lewis Hamilton": "Sir Lewis Carl Davidson Hamilton (born 7 January 1985) is a British racing driver. He currently competes in Formula One for Mercedes, having previously driven for McLaren from 2007 to 2012. In Formula One, Hamilton has won a joint-record seven World Drivers' Championship titles (tied with Michael Schumacher), and holds the records for the most wins (99), pole positions (101), and podium finishes (173), amongst others.",
  "Fernando Alonso": "Fernando Alonso Díaz (born 29 July 1981) is a Spanish racing driver currently competing for Alpine in Formula One. He won the series' World Drivers' Championship in 2005 and 2006 with Renault, having also driven for McLaren, Ferrari and Minardi. With Toyota, Alonso has won the 24 Hours of Le Mans twice, in 2018 and 2019, and the FIA World Endurance Championship in 2018–19, whilst he won the 24 Hours of Daytona with Wayne Taylor Racing in 2019.",
  "Sebastian Vettel": "Sebastian Vettel (born 3 July 1987) is a German racing driver who competes in Formula One for Aston Martin, having previously driven for BMW Sauber, Toro Rosso, Red Bull and Ferrari. Vettel has won four World Drivers' Championship titles which he won consecutively from 2010 to 2013. Vettel is the youngest World Champion in Formula 1. He also has the third most race victories (53) and podium finishes (122) and the fourth most pole positions (57)."}
test_question = "What team does Lewis Hamilton work for?"
qa_model.context_updating(test_context)
qa_model.answer_predicting(test_question)[0]

'mercedes'